# Project milestone 4
# Detection of housing-health relationship


The aim of this project is to estimate the relation between housing quality and health status of a person. The project is an observational study based on a survey made by Mexican National Institute of Statistics and Geography. The health variables reported for each person will be used to define a single health variable which will be a score for each person's health status. We aim to use machine learning methods to do the classification and we will use regression models to predict this health score from the housing variables. Matching will be used to weed out the possible covariates. The motivation is to estimate the most important parameters of housing quality so that we can propose most cost-effective solutions that would increase the quality of health. The original paper is based on the analysis of influence of concrete floors on health quality, while here we would investigate some other parameters such as material used for building and whether there is a toilet or not in the household.

# Step 1: Import data

In [284]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

### Import data ###
#data_household = pd.read_csv('data_translated/household.csv')
data_house = pd.read_csv('data_translated/house.csv',low_memory=False)
data_person = pd.read_csv('data_translated/person.csv',low_memory=False)

In [285]:
#data_person_all = data_person.merge(data_house, left_on='House identifier', right_on='House identifier')
data_person_all = data_person.merge(data_house, left_on=['House_identifier'], right_on=['House_identifier'])

# Step 2: Exploratory Data Analysis

In [286]:
# our selected health variables
health_var = list(data_person_all.columns[35:57]) + [data_person_all.columns[26]]
data_person_all[health_var].describe()

,Difficulty_seeing,Wear_a_hearing_aid,Difficulty_hearing,Difficulty_hearing_without_noise,Difficulty_hearing_with_noise,Dificulty_to_walk,Use_a_walking_device,Walking_apparatus,Difficulty_walking_100_m,Difficulty_walking_500_m,...,Medication_for_nervousness,Intensity_of_nervousness,Frequency_of_depression,Antidepressant_medications,Intensity_of_depression,Frequency_of_pain,Pain_intensity,Fatigue_frequency,Tired_time,Limiting_physical_or_mental_activity
count,208140,208140,208140,208140,208140,208140,208140,208140,208140,208140,...,208140,208140,208140,208140,208140,208140,208140,208140,208140,208140
unique,6,3,6,6,6,6,3,9,6,6,...,4,5,7,4,5,6,5,6,5,2
top,1,2,1,1,1,1,2,,1,1,...,2,,5,2,,1,,1,,
freq,163575,197136,186322,190264,177606,175302,191991,201868,184590,177483,...,178203,117286,126698,180272,152071,132028,157153,129517,154635,205250


In [287]:
# replacing " " values with proper None, what about the 9?
data_person_all = data_person_all.replace(' ', np.nan)

In [288]:
# drop columns with more than 30% of NaN values
nulli = []
exc = []
thr = int(len(data_person_all)*0.3)
for i in range(len(list(data_person_all.columns))):
    nulli.append(data_person_all[list(data_person_all.columns)[i]].isnull().values.sum())
    if (nulli[i] > thr):
        exc.append(i)
data_person_all = data_person_all.drop(columns = [list(data_person_all.columns)[i] for i in exc], axis=1)

In [289]:
# update health variable
health_var = [i for i in health_var if i in list(data_person_all.columns)]
len(health_var)

### Filter nan values

In [293]:
# To do something better just for now maybe fill with mean or so
data_person_all.fillna(0,inplace=True)

### Choose apropriate data type for variables and categorize

In [294]:
# To choose apropriate datatype for each dependant variable or covariates
data_person_all[health_var[0]]=data_person_all[health_var[0]].astype(int)

# Step 3: Regression

### Fill response variables

In [247]:
# To run for all health_var or for health score
response=health_var[0]

### Fill model with all variables

In [296]:
# Do we filter here only important variables for example based on person correlation coef. for each response?

model='~Age'
start=data_person_all.columns.get_loc('Sex')
end=data_person_all.columns.get_loc('Worked_last_week')

for index,item in enumerate(data_person_all.columns):
    # Fill with person (control) variables
    if index >=start and index <=end:
        model=model+'+'+item
        #model=model+'+'+'C('+item+')'

    # To add also House variables
    elif index>end:
        break

In [263]:
formula=""
formula=response+model

In [264]:
# Fits the model (find the optimal coefficients, adding a random seed ensures consistency)
np.random.seed(1950)
mod= smf.ols(formula=response+model, data=data_person_all,missing='raise')
res = mod.fit()
res.summary()